In [15]:
#!/usr/bin/python3
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import xlrd
import pandas as pd



In [16]:
def clean_str(s):  
    s = re.sub('[^0-9\w\d\s]+', '', s).replace('\r'," ").replace('\n'," ").replace('\xa0',' ')
    
    return s.strip().lower()

Train_Fake_Data = []
Train_True_Data = []

f = open("./data/Fake_train")
for line in f.readlines():
    Train_Fake_Data.append(line)
f.close()

f = open("./data/True_train")
for line in f.readlines():
    Train_True_Data.append(line)
f.close()

ALL_word_dict = {}
ALL_word_list = []

for line in Train_Fake_Data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in ALL_word_dict.keys():
            ALL_word_dict[word] = 1
        else:
            ALL_word_dict[word] += 1

for line in Train_True_Data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in ALL_word_dict.keys():
            ALL_word_dict[word] = 1
        else:
            ALL_word_dict[word] += 1


for k, v in ALL_word_dict.items():
    if v > 50:
        ALL_word_list.append(k)

print('ALL Word : ', len(ALL_word_list))



ALL Word :  1381


In [17]:
def data_analysis(target_word_name=None):
    # Data Analysis

    word_list = {}

    #  ************** Target Word List ******************
    print("Get "+target_word_name+" Data Begining ......")
   
    target_words_num = {}
    target_words_list = []
    target_words_weight = {}

    if target_word_name == 'propaganda':

        fake_pro_data = []
        true_pro_data = []
        
        target_data_1 = open('../other_dict/task-1/task1.train.txt', encoding='utf-8', mode='r')
        target_data_1 = target_data_1.readlines()
        for line in target_data_1:
            line = line.strip('\n\r').lower().split('\t')
            text = line[0]
            label = line[2]
            if label == 'propaganda':
                true_pro_data.append(text)
            else:
                fake_pro_data.append(text)
                
        tesk_2_3_path = '../other_dict/tasks-2-3/train/'
        target_data_2_dirs = os.listdir(tesk_2_3_path)
        article_index_set = set()
        for file_name in target_data_2_dirs:
            an_article_index = file_name.split(".")[0] 
            article_index_set.add(an_article_index)
        
        for index in article_index_set:
            text_name = tesk_2_3_path + index + '.txt'
            try:
                text = open(text_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', text_name)
                continue
            try:
                label_name = tesk_2_3_path + index + '.task2.labels'
                label = open(label_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', label_name)
                continue
            text_list = []
            for line in text.readlines():
                text_list.append(line)
            label = label.readlines()
            label_list = []
            for line in label:
                label_list.append(line) 
            #print(len(text_list),len(label_list))
            for i in range(len(text_list)):
                #print(label_list[i])
                if i < len(label_list):
                    if text_list[i].strip() != '':
                        if "\tpropaganda" in label_list[i]:
                            true_pro_data.append(text_list[i])
                        elif "\tnon-propaganda" in label_list[i]:
                            fake_pro_data.append(text_list[i])
                        # else:
                            # print(label_list[i])
                            # print("No Match !!!", index, i)
        print(len(true_pro_data),len(fake_pro_data))

       
        fake_pro_num = 0 
        true_pro_num = 0
        word_list = {}
        for line in fake_pro_data:
            line_word_list = []
            fake_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                word=clean_str(word)
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][0] += 1
                    else:
                        word_list[word] = [1, 0]

        for line in true_pro_data:
            line_word_list = []
            true_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][1] += 1
                    else:
                        word_list[word] = [0, 1]
        
        import nltk
        nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = stopwords.words('english')
        fake_word_dict={}
        true_word_dict={}
        for word, v in word_list.items():
            word_in_fake = v[0]
            word_in_true = v[1]
            if word_in_fake + word_in_true > 10 and word not in stop_words:
                # print(word_in_fake, word_in_true)
                word_not_in_fake = fake_pro_num - word_in_fake
                word_not_in_true = true_pro_num - word_in_true
                obs = np.array([[word_in_fake, word_not_in_fake], [word_in_true, word_not_in_true]])
                g, p, dof, expctd = chi2_contingency(obs)
                if(p < 0.05):  
                    if word_in_fake > word_in_true:  
                        fake_word_dict[word] = p
                    else:
                        true_word_dict[word] = p

        propaganda_word_list = []
        print("fake_word_dict",len(fake_word_dict))
        print("true_word_dict",len(true_word_dict))
        
        for w, p in fake_word_dict.items():
            propaganda_word_list.append(w)
        for w, p in true_word_dict.items():
            propaganda_word_list.append(w)

        for word in propaganda_word_list:
            target_words_num[word] = [0, 0, 0]
            target_words_list.append(word)
            try:
                target_words_weight[word] = '1'
            except:
                target_words_weight[word] = '1'
        
        
        
        
        
        
    #  **************************************************

    fake_data = Train_Fake_Data
    true_data = Train_True_Data


    fake_num = len(Train_Fake_Data) 
    true_num = len(Train_True_Data)

    print('fake_num, true_num:', fake_num, true_num)

    fake_word_list = []
    true_word_list = []

    if target_word_name != 'persuasive':
        for line in fake_data:
            line = line.strip('\n').split(' ')
            for word in line:
                if word not in fake_word_list:
                    fake_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][0] += 1
        print('target_words_num[word][0] += 1 in fake:',target_words_num)
        print('OK_fake_data')
        for line in true_data:
            line = line.strip('\n').split(' ')
            for word in line:
                if word not in true_word_list:
                    true_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][1] += 1
        print('target_words_num[word][1] += 1 in true:',target_words_num)
        print('OK_true_data')


    else:
        for line in fake_data:
            line_1 = line.strip('\n').split(' ')
            for word in line_1:
                if word not in fake_word_list:
                    fake_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][0] += 1
            line_2 = line.strip('\n')
            for phase in persuasive_words_list:
                if phase in line_2:
                    if phase not in fake_word_list:
                        fake_word_list.append(phase)
                    target_words_num[phase][0] += 1

        print('OK_fake_data')
        for line in true_data:
            line_1 = line.strip('\n').split(' ')
            for word in line_1:
                if word not in true_word_list:
                    true_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][1] += 1
            line_2 = line.strip('\n')
            for phase in persuasive_words_list:
                if phase in line_2:
                    if phase not in true_word_list:
                        true_word_list.append(phase)
                    target_words_num[phase][1] += 1
        print('OK_true_data')



    # print(persusive_words_num)
    fake_word_num = len(fake_word_list)
    true_word_num = len(true_word_list)


    fake_word_dict = {}
    true_word_dict = {}

#     print('OK')

   
    for word, v in target_words_num.items():
        word_in_fake = v[0]
        word_in_true = v[1]


        if word_in_fake + word_in_true > 10:
            # print(word_in_fake, word_in_true, word_in_satire)
            word_not_in_fake = fake_num - word_in_fake
            word_not_in_true = true_num - word_in_true

            obs = np.array([[word_in_fake, word_not_in_fake], [word_in_true, word_not_in_true]])
            try:
                chi2, p, dof, expected = chi2_contingency(obs)

                if p < 0.05: 
                    if (word_in_fake > word_in_true) :  
                        fake_word_dict[word] = p
                    elif (word_in_true > word_in_fake) :  
                        true_word_dict[word] = p

            except:
#                 print(word, obs)

                '''
                if word_in_fake > word_in_true:  
                    fake_word_dict[word] = word_in_fake - word_in_true
                elif word_in_fake < word_in_true:
                    true_word_dict[word] = word_in_true - word_in_fake
                else:
                    neutral_word_dict[word] = 0
                '''

    # print(fake_word_dict, true_word_dict, neutral_word_dict)

   
        

    fake_word_dict = sorted(fake_word_dict.items(), key=lambda d: d[1], reverse=False)  
    true_word_dict = sorted(true_word_dict.items(), key=lambda d: d[1], reverse=False)


   
    fake_target_words_path = './result_fake_true_week5/fake_' + target_word_name + '_words.csv'
    with open(fake_target_words_path, "w") as fake_key_words:
        writer = csv.writer(fake_key_words)
        for i, item in enumerate(fake_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])

    true_target_words_path = './result_fake_true_week5/true_' + target_word_name + '_words.csv'
    with open(true_target_words_path, "w") as true_key_words:
        writer = csv.writer(true_key_words)
        for i, item in enumerate(true_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])




In [18]:
data_analysis("propaganda")

Get propaganda Data Begining ......
7959 42297


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


fake_word_dict 11062
true_word_dict 7671
fake_num, true_num: 1921 2893
target_words_num[word][0] += 1 in fake: {'et': [5, 0, 0], '': [6460, 0, 0], 'recent': [72, 0, 0], 'everything': [69, 0, 0], 'find': [72, 0, 0], 'connecticut': [3, 0, 0], 'ended': [30, 0, 0], 'gov': [30, 0, 0], 'gina': [0, 0, 0], 'see': [133, 0, 0], 'least': [53, 0, 0], 'jobs': [62, 0, 0], 'come': [108, 0, 0], 'island': [29, 0, 0], 'certainly': [16, 0, 0], 'less': [53, 0, 0], 'risky': [0, 0, 0], 'state': [170, 0, 0], 'beg': [2, 0, 0], 'pension': [4, 0, 0], 'problems': [25, 0, 0], 'economic': [14, 0, 0], 'issues': [38, 0, 0], 'reported': [151, 0, 0], 'weeks': [50, 0, 0], 'ago': [67, 0, 0], 'islands': [3, 0, 0], 'significant': [11, 0, 0], 'many': [230, 0, 0], 'reflect': [2, 0, 0], 'connecticuts': [0, 0, 0], 'even': [314, 0, 0], 'match': [10, 0, 0], 'prize': [2, 0, 0], '1994': [0, 0, 0], 'within': [39, 0, 0], 'court': [62, 0, 0], 'system': [39, 0, 0], 'exactly': [30, 0, 0], 'gained': [7, 0, 0], 'like': [382, 0, 0], 'inc

target_words_num[word][1] += 1 in true: {'et': [5, 6, 0], '': [6460, 7982, 0], 'recent': [72, 95, 0], 'everything': [69, 86, 0], 'find': [72, 86, 0], 'connecticut': [3, 7, 0], 'ended': [30, 30, 0], 'gov': [30, 43, 0], 'gina': [0, 1, 0], 'see': [133, 235, 0], 'least': [53, 82, 0], 'jobs': [62, 172, 0], 'come': [108, 148, 0], 'island': [29, 45, 0], 'certainly': [16, 34, 0], 'less': [53, 116, 0], 'risky': [0, 4, 0], 'state': [170, 392, 0], 'beg': [2, 5, 0], 'pension': [4, 7, 0], 'problems': [25, 46, 0], 'economic': [14, 54, 0], 'issues': [38, 59, 0], 'reported': [151, 59, 0], 'weeks': [50, 76, 0], 'ago': [67, 110, 0], 'islands': [3, 7, 0], 'significant': [11, 30, 0], 'many': [230, 261, 0], 'reflect': [2, 4, 0], 'connecticuts': [0, 1, 0], 'even': [314, 353, 0], 'match': [10, 13, 0], 'prize': [2, 4, 0], '1994': [0, 6, 0], 'within': [39, 46, 0], 'court': [62, 115, 0], 'system': [39, 86, 0], 'exactly': [30, 38, 0], 'gained': [7, 7, 0], 'like': [382, 623, 0], 'income': [16, 63, 0], 'estate': [

In [19]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
words = stopwords.words('english')
print(words)
import os
path = "./result_fake_true_week5/"
files= os.listdir(path) 
s = []
for file in files: 
     if not os.path.isdir(file): 
        print("now...",file)
        if(file == '.DS_Store'):
            continue
        f = open(path+"/"+file); 
        iter_f = iter(f); 
        fw = open("./result_fake_true_reduce_stopwords_week5/"+file,'w+')
        reduce_list = []
        for line in iter_f: 
            word = line.split(',')[0]
            if word not in words:
                fw.write(line)
            else:
                reduce_list.append(word)
        print(reduce_list)
        f.close()
        fw.close()

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
